In [68]:
# author: Lino Grossano lino.grossano@gmail.com
# author: Stefano Manzini stefano.manzini@gmail.com
from PIL import Image
from pathlib import Path
import glob

In [2]:
image_name = "wallhaven-g78rvl.jpg"
folder_name = "imgs_to_crop"

In [3]:
def load_pic(image_name, folder_name):
    
    """
    Assumes a subdirectory <folder name> containing the
    image <image_name> to load.
    
    params
    ======
    
    image_name: <str>
    folder_name: <str>
    """
    
    fullpath = Path(folder_name, image_name)
    picture = Image.open(fullpath)
    return picture

In [43]:
def crop_in_tiles_and_save(
    image, img_filename=None, tile_size=28, shift=0, img_type="jpg",
    ):
    
    """
    this function crops an image in several tiles; every tile
    is a square of *tile_size* pixels. *shift* = 0 crops the image
    from coordinates (0,0), while *shift* = tile_size/2
    (or other values) crops tiles that overlaps edges.
    Every set of tiles is saved in a separate folder.
    
    
    params
    ======
    
    image: a Pillow open image
    img_filename: <str> name of the input image. If None, we try to get it
    tile_size: <int> pixels; size of the tile side
    shift: <int>: the offset from 0,0 in pixels
    img_type: <str>: desired output filename
    """
    
    assert isinstance(tile_size, int)
    assert isinstance(shift, int)
    assert isinstance(img_type, str)
    
    if img_filename is None:
        try: # for UNIX:
            rev_pos = image.filename[::-1].find("/")
            if rev_pos == -1:
                raise TypeError # we're on Windows            
            pos = len(image.filename) - rev_pos
            img_filename = image.filename[pos:]
        except: # for Windows:
            rev_pos = image.filename[::-1].find("\\")
            pos = len(image.filename) - rev_pos
            img_filename = image.filename[pos:]
        
        try:
            assert "." in img_filename
            assert len(img_filename) > 1
        except:
            print("**Warning**: something went wrong guessing the image filename.")
            print(f"I think this is the image name: {img_filename}")
            print(f"This is the full path: {image.filename}")
            img_filename = "image.jpg"
    
    width, height = image.size
       
    folder = Path(img_filename + "_tiles")
    folder.mkdir(exist_ok=True) #skip errors if folder already exists
    storing_folder = str(folder)

    #calculate coordinates of every tile
    for x in range (0+shift, width, tile_size):
        if width - x < tile_size:
            continue
            
        for y in range (0+shift, height, tile_size):
            if height - y < tile_size:
                continue
            
            # tile coord ===
            tile_coord = (
                x, y, # upper left coords
                x + tile_size, y + tile_size # lower right coords
            )
            
            # output filename and path ===
            xcoord = "%04d" % x
            ycoord = "%04d" % y
            offset = "" if shift == 0 else str(shift)
            outfile_name = f"{img_filename}_{xcoord}_{ycoord}_offset_{offset}.{img_type}"
            full_outpath =  Path(storing_folder, outfile_name)
            
            image.crop(tile_coord).save(full_outpath)

In [9]:
def crop_in_tiles(
    image, tile_size=28, shift=0):
    
    """
    This function crops an image in several tiles
    tile_size × tile_size squares, yielding a tile
    every iteration.
    
    If the input image is not a perfect multiple of
    a(tile_size) × b(tile_size), non-square tiles are NOT
    YIELDED.
    
    params
    ======
    
    image: a Pillow open image
    tile_size: <int> pixels; size of the tile side
    shift: <int>: the offset from 0,0 in pixels
    """
    
    assert isinstance(tile_size, int)
    assert isinstance(shift, int)
    
    width, height = image.size
       
    #calculate coordinates of every tile
    for x in range (0+shift, width, tile_size):
        if width - x < tile_size:
            continue
            
        for y in range (0+shift, height, tile_size):
            if height - y < tile_size:
                continue
            
            # tile coord ===
            tile_coord = (
                x, y, # upper left coords
                x + tile_size, y + tile_size # lower right coords
            )
            
            tile = image.crop(tile_coord)
            yield tile

In [44]:
pic = load_pic(image_name, folder_name)

In [45]:
crop_in_tiles_and_save(pic) # 3942 files (OK!)

In [60]:
class Dataset:
    # TODO: now with 1 image, to be adapted to read all images within a folder
    # and take clear_tag e noise_tag as params

    def __init__(self, img_name, folder_name):
        self.img_name = img_name # to be removed
        self.folder_name = folder_name
        
        self.clean_pics = [self._load_pic(img_name, folder_name)] # TODO
        self.noise_pics = [] # TODO

    
    def _load_pic(self, image_name, folder_name):

        """
        Assumes a subdirectory <folder name> containing the
        image <image_name> to load.

        params
        ======

        image_name: <str>
        folder_name: <str>
        """

        fullpath = Path(folder_name, image_name)
        picture = Image.open(fullpath)
        return picture
    

    def _crop_in_tiles(self, image, tile_size=28, shift=0):

        """
        This function crops an image in several tiles
        tile_size × tile_size squares, yielding a tile
        every iteration.

        If the input image is not a perfect multiple of
        a(tile_size) × b(tile_size), non-square tiles are NOT
        YIELDED.

        params
        ======

        image: a Pillow open image
        tile_size: <int> pixels; size of the tile side
        shift: <int>: the offset from 0,0 in pixels
        """

        assert isinstance(tile_size, int)
        assert isinstance(shift, int)

        width, height = image.size

        #calculate coordinates of every tile
        for x in range (0+shift, width, tile_size):
            if width - x < tile_size:
                continue

            for y in range (0+shift, height, tile_size):
                if height - y < tile_size:
                    continue

                # tile coord ===
                tile_coord = (
                    x, y, # upper left coords
                    x + tile_size, y + tile_size # lower right coords
                )

                tile = image.crop(tile_coord)
                yield tile
        

    def _split_into_channels(self, image):
        return [image.getchannel(x) for x in "RGB"]
    
    
    # for now, only one fixed image and only clean tiles.
    # implement the rest
    def make_dataset(self):
        try:
            assert not hasattr(self, "clean_tiles_")
        except:
            print("Dataset already made.")
            return
        
        # these will store tile1_R, tile1_G, tile1_B, tile2_R, tile2_G, ..
        self.clean_tiles_ = []
        self.noise_tiles_ = []
        
        for clean in self.clean_pics:
            tiles = self._crop_in_tiles(clean, tile_size=56)
            for tile in tiles:
                self.clean_tiles_.extend(self._split_into_channels(tile))
        
        for noise in self.noise_pics:
            tiles = self._crop_in_tiles(noise, tile_size=56)
            for tile in tiles:
                self.noise_tiles_.extend(self._split_into_channels(tile))  